## Importing dependencies

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import load_model
import os 
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Removing unwanted images

In [7]:
data_dir = 'data'

In [8]:
image_extensions = ['jpeg','jpg', 'bmp', 'png']

In [19]:
#looping through the data directories to get all the images and then making sure they are in the right format
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('No issue with image {}'.format(image_path))

No issue with image data/non_potatoes/00IMPORTS-videoSixteenByNine3000.jpg
No issue with image data/non_potatoes/1200px-Marketvegetables.jpg
No issue with image data/non_potatoes/123-fruits-and-vegetables.jpg
No issue with image data/non_potatoes/1260616200_5799399964001_5799397206001-vs.jpg
No issue with image data/non_potatoes/1643560104-b9247ef4-5c68-4cf8-ae35-3268df0d34c4.jpg
No issue with image data/non_potatoes/2-2-2-2foodgroups_vegetables_detailfeature.jpg
No issue with image data/non_potatoes/30-low-maintenance-fruits-vegetables-garden-square.jpg
No issue with image data/non_potatoes/3b360279-8b43-40f3-9b11-604749128187.jpg
No issue with image data/non_potatoes/643188-gettyimages-153946385-ca1ccfaad9be44325afc434b305adc0d.jpg
No issue with image data/non_potatoes/650x350_fun_facts_fruits_and_vegetables_slideshow.jpg
No issue with image data/non_potatoes/a-colorful-selection-of-fruits-and-vegetables.jpg
No issue with image data/non_potatoes/a-spread-of-many-fruits-and-vegetables

## Loading in data with tensorflow

In [20]:
data = tf.keras.utils.image_dataset_from_directory('data')

AttributeError: module 'tensorflow.keras.utils' has no attribute 'image_dataset_from_directory'